# 1. 엑셀데이터 불러오기

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re


df_input = pd.read_csv('phone_info.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

# 2. 리뷰크롤링

In [32]:
from selenium.webdriver import ActionChains
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델영문명
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": ""                        #리뷰내용 
}

crawl_data = pd.DataFrame(my_dict,index=[0])
crawl_data_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    else:
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000204&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_input.iloc[index][4]+url2)
    driver.implicitly_wait(3)
    
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴 => 5개 넘어가면 검색정확도 떨어져서 고정
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
            pass
        else:
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") 
            market = area1[3].text 
            write_id = area1[4].text 
            upload_day = area1[5].text 
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            my_dict = {
                "pl_id": df_input.iloc[index][0],              #기종아이디(자체부여한것)
                "pl_model_code": df_input.iloc[index][3],      #모델코드
                "pl_name": df_input.iloc[index][4],            #모델영문명

                "star": star,                       #별점
                "market": market,                   #구입처(ex 인터파크, 11번가)
                "write_id": write_id,               #작성자(일부가림처리)
                "upload_day": upload_day,           #업로드날짜
                "title":title,                      #리뷰제목 
                "text": text,                       #리뷰내용 
            }
            
            if my_dict is None or my_dict == None:
                pass
            else:
                a = pd.DataFrame(my_dict,index=[0])
                crawl_data = crawl_data.append(a)
                crawl_data = crawl_data.reset_index(drop=True)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    #display(crawl_data)
    return crawl_data

# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)


# for i in range(len(df_input)):
for i in range(10,11):
#     print("=================",i,"skt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"skt")) 
#     print("=================",i,"kt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"kt"))
#     print("=================",i,"lg","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"lg"))
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"all"))
    crawl_data = crawl_data[crawl_data['pl_id'] != '']
    
    

,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text
0,,,,,,,,,


In [33]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
display(crawl_data)

pandas version:  1.0.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text
1,11,A1784,아이폰7 Plus,5,G마켓,swee****,17.05.19.,알찬구매기,아이폰7플러스RED갤럭시 s6 엣지플러스에서 갈아탄 내생애 첫아이폰이라 의미가 더 ...
2,11,A1784,아이폰7 Plus,4,인터파크,fba*****,16.12.15.,드디어 도착했네요.!,요청후 재고가 없어 조금 기다리긴 했지만 기다린 만큼 잘 받았습니다. 다만. 같이 ...
3,11,A1784,아이폰7 Plus,4,인터파크,vlr*****,16.12.05.,잘받았습니다!!,아이폰7 플러스를 찾던 도중 갑자기 알림오는 브랜드딜로 적당한 가격에 매트블랙을 잘...
4,11,A1784,아이폰7 Plus,5,옥션,m*0*****,19.12.31.,단종색상 득템 감사,단종색상 득템 감사
5,11,A1784,아이폰7 Plus,5,G마켓,whty****,17.06.15.,좋아요~,결제하고 핸드폰 받기까지 4일정도 걸린거 같아요~ 만족합니다
6,11,A1784,아이폰7 Plus,5,인터파크,soji****,17.09.24.,좋아요,신랑 사줬는데 삼성쓰다가 넘어간거라 아직 헤매고있긴하지만 좋아요 ㅎ ㅎ
7,11,A1784,아이폰7 Plus,5,인터파크,kore****,17.08.12.,만족,믿고 만족합니다 물건잘잗았습니다
8,11,A1784,아이폰7 Plus,5,G마켓,jise****,17.03.28.,좋아요,배송이빠르고 상담 친절해용
9,11,A1784,아이폰7 Plus,5,G마켓,cd4e****,17.02.09.,만족합니다,오프라인 대비 저렴하게 구입해서 좋습니다. 원하는 컬러 및 사은품 반영되어 주문후 ...
10,11,A1784,아이폰7 Plus,5,인터파크,nala****,17.04.28.,괜찮네요,"여기서 아이폰7 레드 구매했습니다에누리에서 보고 할인받아서 싸게 샀는데, 현대카드로..."


In [18]:
crawl_data.to_csv('crawl_data.csv', # file path, file name
                  sep=',')   # seperator, delimiter (구분자)
#     ...: na_rep='NaN')   # missing data representation (결측값 표기)
crawl_data.to_csv('file.csv',encoding='utf-8-sig')



# 불러온 엑셀데이터로 차례대로 파싱

In [36]:
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

In [76]:
crawl_data.to_csv('crawl_data.csv', # file path, file name
                  sep=',')   # seperator, delimiter (구분자)
#     ...: na_rep='NaN')   # missing data representation (결측값 표기)

